# Command Line version: 

### Estimate response function for spherical deconvolution (outputs out.txt)
dwi2response tournier denoise.mif out.txt

### Estimate fibre orientation distributions (outputs fod.mif)
dwi2fod csd denoise.mif out.txt fod.mif -mask mask.nii

### Perform streamlines tractography (outputs tract.tck)
tckgen fod.mif tract.tck -seed_image mask.nii -mask mask.nii -select 10000000 

### Use track data to produce high-resolution image (outputs tdi.nii)
tckmap tract.tck tdi.nii -template b0.nii

In [ ]:
import os
os.environ['PATH'] += ':/Users/christianraimondo/ants-2.5.0/bin'
nthreads = os.environ.get('MRTRIX_NTHREADS', 'Not set')
print(f'MRTRIX_NTHREADS is {nthreads}')

import tempfile
import subprocess
from tqdm import tqdm
import numpy as np
import nibabel as nib
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.reconst.dti import gradient_table, TensorModel
from dipy.reconst.dti import fractional_anisotropy, mean_diffusivity, color_fa
from dipy.core.gradients import gradient_table
import nibabel as nib
from dipy.reconst.dti import TensorModel, mean_diffusivity, decompose_tensor
import shutil   
from dipy.io import read_bvals_bvecs


def run_command(command, step_name):
    print(f"Running: {step_name}")
    result = subprocess.run(command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        print(f"Command failed with code {result.returncode}: {result.stderr}")
    return result


def tracto(patient_dir):
    steps = ["Estimating response function for spherical deconvolution", "Estimating fibre orientation distributions", "Performing streamlines tractography", "Generating final image"]
    
    input_dir = os.path.join(patient_dir,"dti_outputs" )
    output_dir = os.path.join(patient_dir, "tractography_outputs")
    os.makedirs(output_dir, exist_ok=True)

    denoise_path = os.path.join(input_dir, "dti_denoise_preproc_unbiased.mif")
    
    for i, step_name in enumerate(tqdm(steps, desc="Processing Tractography", unit="step")):
        if step_name == "Estimating response function for spherical deconvolution":
            run_command(f"dwi2response dhollander {denoise_path} {output_dir}/wm_response.txt {output_dir}/gm_response.txt {output_dir}/csf_response.txt -mask {input_dir}/mask.nii -fslgrad {input_dir}/bvec.bvec {input_dir}/bval.bval", step_name)

        elif step_name == "Estimating fibre orientation distributions":
            run_command(f"dwi2fod msmt_csd {denoise_path} {output_dir}/out.txt {output_dir}/wmfod.mif {output_dir}/response_gm.txt {output_dir}/gm.mif {output_dir}/response_csf.txt {output_dir}/csf.mif -mask {input_dir}/mask.nii -fslgrad {input_dir}/bvec.bvec {input_dir}/bval.bval", step_name)
        
        elif step_name == "Performing streamlines tractography":
            run_command(f"tckgen {output_dir}/wmfod.mif {output_dir}/tract.tck -seed_image {input_dir}/mask.nii -mask {input_dir}/mask.nii -select 10000000 -fslgrad bvec.bvec bval.bval", step_name)
        
        elif step_name == "Generating final image":
            run_command(f"tckmap {output_dir}/tract.tck {output_dir}/tdi.nii -template {input_dir}/b0.nii", step_name)

#this is the folder with all of the pt folders in it
base_path = "/media/jimric/Mahdi-2022/Laxman_PD_Project/MAP_complete/"

patient_list = os.listdir(base_path)

for patient in patient_list:
    patient_dir = os.path.join(base_path, patient)
    try:
        print(f"starting processing of {patient}.")
        tracto(patient_dir)
        print(f"{patient_dir} processing completed successfully.")
    except Exception as e:
        print(f"Error while processing {patient}: {str(e)}")